In [1]:
import GUI
import nltk
from nltk.stem.snowball import SnowballStemmer  # type: ignore
from nltk.corpus import wordnet as wn
import numpy as np # type: ignore
import pandas as pd # type: ignore
import re
import os

In [2]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/glados/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# function to create the excel files
def create_excel_dataframe(input , name, columns, index):
    df = pd.DataFrame(input, columns = columns, index = index) # Create a pandas dataframe
    df.to_excel(name) # Create an excel file
    return df

In [4]:
# read the list of stop words
stopword_file_path = "/home/glados/Documents/AmirAli Toori/Lessons/Python/IR-Project/gist_stopwords.txt"
with open(stopword_file_path, "r") as file:
    stop_words = file.read()

In [5]:
def convert_to_list(doc) -> list:
    doc = re.sub(r"[^\w\s]", " ", doc)
    doc = doc.split()
    return doc

In [6]:
stemmer = SnowballStemmer(language='english')
dictionary = []

In [7]:
def is_meaningful(word) -> bool:
    
    if wn.synsets(word):
        return True
    else: 
        return False

In [8]:
def create_dictionary(doc, dictionary) -> list:
    for word_index, word in enumerate(doc):
        
        word = word.lower()
        
        if word.isalnum() and word not in stop_words and is_meaningful(word):
            
            stem_word = stemmer.stem(word)
            doc[word_index] = stem_word
            
            if stem_word not in dictionary:
                dictionary.append(stem_word)
    
    return doc

In [9]:
def join_document(doc) -> str:
    doc = ' '.join(doc)
    return doc

In [10]:
def process_document(document) -> str:
    
    document = convert_to_list(document)
    
    document = create_dictionary(document, dictionary)
    
    document = join_document(document)
    
    return document

In [11]:
raw_document = GUI.select_file() # Select File GUI implementation

raw_dataset = raw_document # the raw dataset that read from the one file

raw_dataset = raw_dataset.split(".I") # split the docs by index
raw_dataset.remove(raw_dataset[0]) # remove the empty first index

doc_index = 0

# write each index in 'raw_dataset'
for doc in raw_dataset:
    doc_index += 1
    doc_path = "/home/glados/Documents/AmirAli Toori/Lessons/Python/IR-Project/docs" + "/doc" + str(doc_index) + ".txt"
    doc = process_document(doc)
    
    if not os.path.exists(doc_path):
        with open(doc_path, "w") as document: # write each index into the separated doc
            document.write(doc)

doc_index = 0

for doc in raw_dataset:
    doc_index += 1
    doc_path = "/home/glados/Documents/AmirAli Toori/Lessons/Python/IR-Project/Original docs" + "/docs" + str(doc_index) + ".txt"
    
    if not os.path.exists(doc_path):
        with open(doc_path, "w") as document: # write each index into the separated doc
            document.write(doc)

GUI.custom_window("700x35", "Extract Documents", f"There have been {doc_index} documents are extracted.")

In [25]:
df = pd.DataFrame(dictionary, columns=["Words"])
df

,Words
0,experiment
1,investig
2,aerodynam
3,scs
4,25
...,...
3559,reformul
3560,delin
3561,hopkin
3562,aeolotrop


In [26]:
df.to_excel("dictionary.xlsx")

In [27]:
term_doc_matrix = np.zeros((len(dictionary), doc_index))
tf_array = np.zeros((len(dictionary), doc_index))
idf_array = np.zeros((len(dictionary), 1))
tf_idf_array = np.zeros((len(dictionary), doc_index))

In [28]:

def calculate_term_frequency(doc, dictionary, doc_no):
    for word in dictionary:
        if word in doc:
            frequency = doc.count(word)
            row = dictionary.index(word)
            column = doc_no - 1
            tf_array[row, column] += frequency


def calculate_inverse_document_frequency(doc, dictionary):
    for word in dictionary:
        if word in doc:
            row = dictionary.index(word)
            idf_array[row, 0] += 1

In [29]:
doc_names_list = []
for num in range(1, doc_index + 1):
    doc_path = "/home/glados/Documents/AmirAli Toori/Lessons/Python/IR-Project/docs" + "/doc" + str(num) + ".txt"
    doc_names_list.append("doc" + str(num) + ".txt")
    with open(doc_path, "r") as file:
        doc = file.read()
        calculate_term_frequency(doc, dictionary, num)
        calculate_inverse_document_frequency(doc, dictionary)



In [30]:
# calculating tf array
row, column = np.shape(tf_array)
for r in range(row):
    for c in range(column):
        if tf_array[r, c] > 0:
            extracted_element = tf_array[r, c]
            tf_array[r, c] = 1 + np.log10(extracted_element)
        else:
            tf_array[r, c] = 0
            
GUI.custom_window("700x35", "Complete!", "The calculation of Term Frequency is have been successful.")

In [31]:
count_of_documents = column
row, column = np.shape(idf_array)


# calculating idf array
for r in range(row):
    if idf_array[r, 0] > 0:
        extracted_element = idf_array[r, 0]
        idf_array[r, 0] = np.log10(count_of_documents / extracted_element)
    else:
        idf_array[r, 0] = 0

GUI.custom_window("700x35", "Complete!", "The calculation of Inverse Document Frequency is have been successful.")

In [32]:
# multiply peer to peer each row of tf-array in idf-array
row, column = np.shape(tf_idf_array)
r, c = 0, 0
while r < row:
        tf_idf_array[r, c]= tf_array[r, c] * idf_array[r, 0]
        c += 1
        if c == column:
            r += 1
            c = 0

In [33]:

tf_dataframe = create_excel_dataframe(tf_array, "tf_excel.xlsx", doc_names_list, dictionary)

idf_dataframe = create_excel_dataframe(idf_array, "idf_excel.xlsx", ["IDF"], dictionary)

tf_idf_dataframe = create_excel_dataframe(tf_idf_array, "tf_idf_excel.xlsx", doc_names_list, dictionary)

GUI.custom_window("700x35", "Excel File", "Creating the Excel file is DONE!")

In [34]:
tf_dataframe

,doc1.txt,doc2.txt,doc3.txt,doc4.txt,doc5.txt,doc6.txt,doc7.txt,doc8.txt,doc9.txt,doc10.txt,...,doc1391.txt,doc1392.txt,doc1393.txt,doc1394.txt,doc1395.txt,doc1396.txt,doc1397.txt,doc1398.txt,doc1399.txt,doc1400.txt
experiment,1.477121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
investig,1.301030,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.477121,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
aerodynam,1.301030,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
scs,1.000000,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
25,1.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
reformul,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
delin,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
hopkin,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aeolotrop,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
idf_dataframe

,IDF
experiment,0.561797
investig,0.588621
aerodynam,0.878956
scs,0.614649
25,1.133291
...,...
exot,3.146128
shelter,3.146128
midway,3.146128
regist,3.146128


In [51]:
tf_idf_dataframe

,doc1.txt,doc2.txt,doc3.txt,doc4.txt,doc5.txt,doc6.txt,doc7.txt,doc8.txt,doc9.txt,doc10.txt,...,doc1391.txt,doc1392.txt,doc1393.txt,doc1394.txt,doc1395.txt,doc1396.txt,doc1397.txt,doc1398.txt,doc1399.txt,doc1400.txt
experiment,0.829842,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.561797,0.000000,0.000000,0.000000,0.561797,0.561797,0.0,0.0,0.000000
investig,0.765813,0.588621,0.0,0.000000,0.000000,0.000000,0.588621,0.588621,0.869464,0.000000,...,0.000000,0.000000,0.588621,0.000000,0.588621,0.000000,0.000000,0.0,0.0,0.588621
aerodynam,1.143549,0.000000,0.0,0.000000,0.878956,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.878956,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
scs,0.614649,0.000000,0.0,0.614649,0.614649,0.614649,0.614649,0.614649,0.614649,0.614649,...,0.000000,0.000000,0.614649,0.614649,0.000000,0.000000,0.000000,0.0,0.0,0.000000
25,1.133291,0.000000,0.0,0.000000,0.000000,1.133291,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
exot,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
shelter,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
midway,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
regist,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
